In [2]:
import pandas as pd
import numpy as np
from huggingface_hub import login, logout
from datasets import load_dataset
from util.util_data import tokenize
import requests, os, sys, sqlite3, re
import yfinance as yf

In [8]:
eng = sqlite3.connect('./src/data.db')
tix = pd.read_sql('SELECT DISTINCT ticker FROM catalysts', eng)
df = pd.read_sql('SELECT * FROM catalysts', eng)
tickers = yf.Tickers(' '.join(tix.ticker.tolist()))

In [33]:
names = list()
for t in tix.ticker.tolist():
    try:
        name = tickers.tickers[t].info['longName'].upper()
        name = re.sub(r'[^\w\s]', '', name)
        names.append([t, name])
    except Exception:
        names.append([t, None])

In [4]:
import json

with open('./ref/ticker_to_cik.json') as f:
    d = json.load(f)
f.close()

In [29]:
ticks = pd.DataFrame.from_dict(d, orient='index')
ciks = ticks.loc[ticks.index.isin(tix.ticker)].copy().reset_index().rename(columns={'index': 'ticker', 0: 'cik'})
ciks['cik'] = ciks.cik.str.removeprefix('000')

In [31]:
ciks.to_csv('./out/ciks.csv', index=False)

In [72]:
url = f'https://www.sec.gov/Archives/edgar/full-index/2009/QTR1/master.idx'
lines = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}, stream=True).content.decode("utf-8", "ignore").splitlines()

In [39]:
SECDIR = 'https://www.sec.gov/Archives'
s = lines[55].split('|')
file_url = os.path.join(SECDIR, s[-1])
response = requests.get(file_url, stream=True, headers= {'User-Agent': 'Mozilla/5.0'})

In [73]:
recs = pd.DataFrame([line.split('|') for line in lines], columns=['cik', 'name', 'filing', 'date', 'url']).iloc[11:]

In [74]:
y = recs.loc[(recs.cik.isin(ciks.cik)) & (recs.filing == '8-K')].copy()
y.cik = y.cik.astype(str)
ciks.cik = ciks.cik.astype(str)

In [75]:
y

,cik,name,filing,date,url
431,1000694,NOVAVAX INC,8-K,2009-01-05,edgar/data/1000694/0000950133-09-000002.txt
432,1000694,NOVAVAX INC,8-K,2009-01-13,edgar/data/1000694/0000950133-09-000041.txt
433,1000694,NOVAVAX INC,8-K,2009-01-15,edgar/data/1000694/0000950133-09-000053.txt
434,1000694,NOVAVAX INC,8-K,2009-02-09,edgar/data/1000694/0000950133-09-000252.txt
435,1000694,NOVAVAX INC,8-K,2009-02-17,edgar/data/1000694/0000950133-09-000346.txt
...,...,...,...,...,...
157128,1390478,RXI PHARMACEUTICALS CORP,8-K,2009-01-07,edgar/data/1390478/0000950135-09-000101.txt
157129,1390478,RXI PHARMACEUTICALS CORP,8-K,2009-01-22,edgar/data/1390478/0000950135-09-000375.txt
157130,1390478,RXI PHARMACEUTICALS CORP,8-K,2009-01-29,edgar/data/1390478/0000950135-09-000543.txt
157131,1390478,RXI PHARMACEUTICALS CORP,8-K,2009-02-05,edgar/data/1390478/0000950135-09-000646.txt


In [70]:
furl = os.path.join(SECDIR, y.iloc[0].url)
response = requests.get(furl, stream=True, headers= {'User-Agent': 'Mozilla/5.0'})

In [81]:
from read8k import clean8k

with open('./out/8Ks/ACOR_2009_QTR1_8K.txt', 'r', encoding='UTF-8', errors='ignore') as fin:
    doc = fin.read()
fin.close()
clean = clean8k(doc=doc)

In [83]:
outdir = 'out/8Ks'
os.listdir(os.path.join(os.getcwd(), outdir))

['NVAX_2009_QTR1_8K.txt',
 'CERS_2009_QTR1_8K.txt',
 'AIMD_2009_QTR1_8K.txt',
 'PLX_2009_QTR1_8K.txt',
 'SGMO_2009_QTR1_8K.txt',
 'ACOR_2009_QTR1_8K.txt',
 'RIGL_2009_QTR1_8K.txt',
 'DVAX_2009_QTR1_8K.txt',
 'RMTI_2009_QTR1_8K.txt',
 'TGTX_2009_QTR1_8K.txt',
 'INO_2009_QTR1_8K.txt',
 'ANIP_2009_QTR1_8K.txt',
 'BMRN_2009_QTR1_8K.txt',
 'ELOX_2009_QTR1_8K.txt',
 'SEEL_2009_QTR1_8K.txt',
 'SIGA_2009_QTR1_8K.txt']